# <font color=#cd0000> Propósito principal </font>
- La idea de este librillo es preparar el entorno para realizar pruebas para cualquier DataSet a ser clasificado por cualquier codificación aplicada a LSTM

## <font color=#cd0000> Leyenda </font>
- Los apartados titulados con el código de colores HEX: `#cd0000` serán apartados que tendrán todos los librillos, en concreto, aquellos especificados en el apartado `Síntesis de los criterios usados` del trabajo.
- Los apartados titulados con el código de colores HEX: `#2451ff` serán apartados de conclusiones propias de este librillo resultado de aplicar un estudio personalizado para cada planteamiento.

# <font color=#cd0000> Prerrequisitos </font>
## <font color=#cd0000> Entorno de ejecución </font>
- Cambiamos el directorio raíz del librillo para acceder cómodamente a las funciones de utilidad.

In [ ]:
import os

os.chdir('../..')
os.listdir()


## <font color=#cd0000> Constantes y variables predefinidas </font>

In [ ]:
HEARTBEAT_PATH = "data/heartbeat"
EPILEPSY_PATH = "data/epilepsy"
SEGUIMIENTO_OCULAR_PATH = "data/seguimiento-ocular/Data/Hospital"
SEGUIMIENTO_OCULAR_FOLDERS_ID = range(1, 12+1)

DATA_TO_SAVE = "HeartBeat"

PKL_DIR = "pkl/<classifier>/<ds>/"
PKL_NAME = "<ds>_<classifier>_<codif>.pkl"

SEED = 1


# <font color=#cd0000> Carga del Dataset </font>

In [ ]:
# TODO - Change with known data
from utils.load_data import import_epilepsy_dataset

# train, test = import_heartbeat_dataset(HEARTBEAT_PATH)
# all_data = import_seguimiento_ocular_dataset(SEGUIMIENTO_OCULAR_PATH)


In [ ]:
# import pickle

# pickle.dump(train, open(DATA_TO_SAVE + "_tmp_train_data.pkl", 'wb'))
# pickle.dump(test, open(DATA_TO_SAVE + "_tmp_test_data.pkl", 'wb'))


In [ ]:
# import pickle

# train = pickle.load(open(DATA_TO_SAVE + "_tmp_train_data.pkl", 'rb'))
# test = pickle.load(open(DATA_TO_SAVE + "_tmp_test_data.pkl", 'rb'))


In [ ]:
train.reset_changes()
test.reset_changes()


## <font color=#cd0000> Particionado inicial de los datos si fuera necesario </font>

In [ ]:
# from utils.data_extraction import Data

# X_train_Data, X_test_Data, y_train, y_test = all_data.train_test_split(
#     criterion='windowed',
#     train_size=.8,
#     random_state=SEED,
#     drop_columns=[]
# )

# X_train_Data = Data(X_train_Data)
# X_test_Data = Data(X_test_Data)

# <font color=#cd0000> Preprocesamiento </font>

## <font color=#cd0000> Eliminación de datos inválidos y valores atípicos </font>
- TODO: Breve descripción de qué es un dato inválido (-1's en columna, etc.)
- Eliminaremos aquellos valores fuera de los percentiles 5 y 95.
- TODO: Definiremos cuál será el límite de outliers permitido por serie temporal

In [ ]:
# TODO - Remove invalid data

In [ ]:
train.remove_outliers(
    headers=train.get_derived_data_columns()['attrs'],
    outliers_limit=.3
)

test.remove_outliers(
    headers=test.get_derived_data_columns()['attrs'],
    outliers_limit=.3
)


In [ ]:
import pandas as pd

# Remaining series
print("Train: Previous number of series: {}".format(
    len(pd.unique(train.original_data['id']))))
print("Train: Actual number of series: {}".format(
    len(pd.unique(train.derived_data['id']))))

print("Test: Previous number of series: {}".format(
    len(pd.unique(test.original_data['id']))))
print("Test: Actual number of series: {}".format(
    len(pd.unique(test.derived_data['id']))))


## <font color=#cd0000> Resoluciones a aplicar </font>
- TODO:
  - Si las series son rápidas (muchos cambios en poco tiempo) especificar resoluciones altas (sin modificaciones).
  - Si las series son lentas (pocos cambios en mucho tiempo) especificar resoluciones bajas (eliminamos datos).

In [ ]:
# Series lentas
train.reduce_sampling_rate(remove_one_each_n_samples=2)
test.reduce_sampling_rate(remove_one_each_n_samples=2)


# <font color=#cd0000> Codificación </font>

In [ ]:
# TODO

## <font color=#cd0000> Normalización </font>

In [ ]:
from utils.codifications import standardize_data

train.derived_data, test.derived_data = standardize_data(
    train.derived_data,
    test.derived_data,
    headers=train.get_derived_data_columns()['attrs']
)


## <font color=#cd0000> Tensores entrada y salida de la red </font>

In [ ]:
# All lengths must be equal
series_length = train.get_shortest_serie().shape[0]
n_dims = len(train.get_derived_data_columns()['attrs'])

(
    train.get_shortest_serie().shape[0],
    train.get_largest_serie().shape[0],
    test.get_shortest_serie().shape[0],
    test.get_largest_serie().shape[0]
)


In [ ]:
import pandas as pd

# This will determine the number of series of each split
train_n_series = pd.unique(train.derived_data['id']).shape[0]
test_n_series = pd.unique(test.derived_data['id']).shape[0]

(train_n_series, test_n_series)


In [ ]:
X_train, _ = train.transform_derived_data_into_X_y()
X_test, _ = test.transform_derived_data_into_X_y()

y_train = train.derived_data.groupby('id').first()['class'].to_numpy()
y_test = test.derived_data.groupby('id').first()['class'].to_numpy()


## <font color=#cd0000> Técnicas de balanceo </font>

### <font color=#cd0000> Asignación de pesos a las clases </font>

In [ ]:
import numpy as np
from sklearn.utils import compute_class_weight

class_weights = compute_class_weight(
    'balanced', classes=np.unique(y_train), y=y_train)
class_weights = {0: class_weights[0], 1: class_weights[1],
                 2: class_weights[2], 3: class_weights[3]}


In [ ]:
from utils.classifier_utils import apply_lstm_format
from sklearn.preprocessing import LabelEncoder

sequences_fragmenter = 2

enc = LabelEncoder()
enc.fit(y_train)

X_train, y_train = apply_lstm_format(
    X_train, y_train, train_n_series, series_length, sequences_fragmenter, enc)
X_test, y_test = apply_lstm_format(
    X_test, y_test, test_n_series, series_length, sequences_fragmenter, enc)


# <font color=#cd0000> Diseño de la topología de red </font>

## <font color=#cd0000> Preparación de mecanismo argmax en caso de salida multiclase </font>

In [ ]:
import numpy as np
import keras.backend as K


def argmax(x, n_classes):
    all_predictions = []
    for max_class_position in K.argmax(x):
        prediction = np.zeros(n_classes)
        prediction.put(max_class_position, 1)
        all_predictions.append(prediction)
    return np.asarray(all_predictions)


In [ ]:
import keras as k
from keras.models import Sequential
from keras.layers import LSTM, Dense

nn = Sequential()

# Number of initial dimensions
nn.add(LSTM(units=12, dropout=.2, recurrent_dropout=.2))
# Number of Epilepsy's classes
nn.add(Dense(4, activation='sigmoid'))

## <font color=#cd0000> Compilación de la red </font>

In [ ]:
from keras.optimizers import RMSprop

metrics = [
    k.metrics.CategoricalAccuracy(name="ACC"),
    k.metrics.Precision(name='Prec'),
    k.metrics.Recall(name='Rec'),
    k.metrics.AUC(name='AUC')
]

nn.compile(optimizer=RMSprop(
    learning_rate=1e-4), loss='binary_crossentropy', metrics=metrics)
nn.build(input_shape=X_train.shape)


## <font color=#cd0000> Visualización de resultados preliminares </font>

In [ ]:
import matplotlib.pyplot as plt


def show_metrics(history):
    for metric in history.history.keys():
        if not metric.startswith('val_'):
            plt.plot(history.history[metric], label=metric)
            plt.plot(history.history[f'val_{metric}'], label=f'val_{metric}')
            plt.title(metric)
            plt.ylabel('')
            plt.xlabel('Epoch')
            plt.legend(loc="upper left")
            plt.show()


## <font color=#cd0000> Entrenamiento del modelo </font>

In [ ]:
import numpy as np
from keras import backend as K
from sklearn.metrics import confusion_matrix, classification_report

epochs = 10

history = nn.fit(X_train, y_train, epochs=epochs,
                 validation_data=(X_test, y_test),
                 class_weight=class_weights,
                 verbose=1)
nn.summary()
print('\n\n')

y_pred = argmax(nn.predict(X_test), 4)

show_metrics(history)

## <font color=#cd0000> Clasificación </font>

In [ ]:
y_real = enc.inverse_transform(y_test)
y_pred = enc.inverse_transform(y_pred)
print(confusion_matrix(y_real, y_pred))
print(classification_report(y_real, y_pred, zero_division=0))

## <font color=#cd0000> Discusión de los resultados </font>
- Dimensionalidad de entrada inicial -> TODO
- Vamos a elegir **diferentes funciones de activación**, **divisiones en ventanas de cada serie del conjunto de datos** (valor ``sequence_fragmenter`` del apartado ``Tensores de entrada y salida de la red``), **diferentes funciones de pérdida**, **optimizadores**, **regularizadores y Dropouts** en caso de sobreajuste, **número de capas ocultas** y, finalmente, **diferentes unidades de neuronas por capa oculta** y **la tasa de aprendizaje** (**estas dos últimas a optimizar** mediante una búsqueda aleatoria de hiper-parámetros).

### <font color=#cd0000> Funciones de activación de las capas ocultas </font>
- TODO: (lo más común es lo siguiente) Dado que tratamos con una entrada de **valores continuos** sería interesante utilizar una **función de activación ReLU** para las capas ocultas junto con una **inicialización de pesos ReLU** (``He_uniform`` en Keras).

### <font color=#cd0000> Función de pérdida y capa de salida </font>
- TODO: Al enfrentarnos a un **problema de clasificación binaria** la capa de **salida contendrá una sola neurona** con una **función de activación sigmoidal** con un umbral de ``0.5`` para determinar si es de una clase u otra (si la activación supera el umbral será de clase ``normal`` y ``abnormal`` de lo contrario).
- TODO: En cuanto a **la función de pérdida** y dado que el problema es de clasificación binaria usaremos **la función ``binary_crossentropy``** de Keras.

### <font color=#cd0000> Tamaño de ventana y número de ventanas de cada serie del conjunto de datos </font>
- \#TODO: ¿Por qué dividirlo en varias ventanas proporciona una mejor precisión al clasificador?

### <font color=#cd0000> Optimizadores y número de capas ocultas </font>
- **Probaremos inicialmente con un número de neuronas en las capas ocultas igual al de la dimensión de entrada** y con **una tasa de aprendizaje brindada por la siguiente expresión: $\frac{0.1}{nº\_ejemplos\_train}$**
- En cuanto a los optimizadores probaremos el desempeño de los siguientes junto con diferentes arquitecturas de red:
  - Adam:
    - Red de 1 capa:
    - Red de 2 capas:
  - RMSprop
    - Red de 1 capa:
    - Red de 2 capas:
  - SGD
    - Red de 1 capa:
    - Red de 2 capas:

### <font color=#cd0000> Neuronas y tasa de aprendizaje </font>
- **Para determinar cuáles son los rangos de neuronas y tasa de aprendizaje en los que buscar los mejores** escogeremos los mejores modelos encontrados hasta ahora y probaremos con diferentes codificaciones: (fan-in -> La mitad de neuronas que la capa anterior; fan-out -> el doble de neuronas que la capa anterior; tasa de aprendizaje base la hallada mediante la expresión: $\frac{0.1}{nº\_ejemplos\_train}$)
  - Fan-in + tasa aprendizaje más pequeña a la base
  - Fan-out + tasa aprendizaje más pequeña a la base
  - Fan-in + tasa aprendizaje más grande que la base
  - Fan-out + tasa aprendizaje más grande que la base
  - Fan-in + tasa aprendizaje regular
  - Fan-out + tasa aprendizaje regular

### <font color=#cd0000> Regularizadores y capa de Dropout </font>
- **En caso de incurrir en sobreajuste** (algo que visualizaremos a posteriori) **probaremos a utilizar una capa de Dropout** con una tasa de no reajuste de pesos del ``0.2``, **de no conseguir resultados, utilizaremos regularizadores ``L1L2``** con ambos factores de regularización a ``0.01`` sobre todas las capas ocultas.
- Tras realizar algunas pruebas observamos que hay sobreajuste y, para resolverlo, utilizaremos la capa de Dropout mencionada en el párrafo anterior.

## <font color=#cd0000> Conclusiones </font>
- Como podemos observar ...
- No obstante si no tuviéramos más remedio que utilizarlo de esta forma buscaríamos el mejor en el rango orientativo:
  - `n_units`: [...]
  - `learning_rate`: [...]

# <font color=#cd0000> Randomized Search </font>
- Búsqueda de hiper-parámetros aleatoria con LSTM maximizando ``macro avg f1-score``

## <font color=#cd0000> Rangos de búsqueda </font>
- Como vimos anteriormente los rangos de búsqueda aleatoria de los mejores hiper-parámetros serán los siguientes

In [ ]:
N_UNITS_RANGE = TODO
LEARNING_RATE_CHOICES = TODO

In [ ]:
import random
import pickle
import utils.constants as cs
from utils.classifier_utils import (windowed_cross_val,
                                    compute_classification_reports_means)
from utils.plot_utils import pretty_print_classification_report_dict
from keras.layers import LSTM, Dense
from keras.optimizers import RMSprop


def lstm_randomized_search_cv(
        windowed_series,
        relation_with_series,
        prefix,
        cv=5,
        lstm_dict={}):
    global PKL_DIR
    all_clf_used = {}

    n_samples = 5
    units_list = random.sample(
        list(N_UNITS_RANGE), n_samples)
    learning_rate_list = random.sample(
        LEARNING_RATE_CHOICES, n_samples)

    best_hyp_params = None
    best_score = 0
    for units in units_list:
        for learning_rate in learning_rate_list:
            lstm_dict[cs.LSTM_HYP_PARAM_UNITS] = units
            lstm_dict[cs.LSTM_HYP_PARAM_LEARNING_RATE] = learning_rate
            reports = windowed_cross_val(
                None,
                windowed_series,
                relation_with_series,
                cv=cv,
                seed=SEED,
                drop_columns=['class'],
                estimator_type=cs.ESTIMATOR_LSTM,
                lstm_dict=lstm_dict
            )

            mean_report = compute_classification_reports_means(reports)
            all_clf_used[(units, learning_rate)] = mean_report

            if mean_report['macro avg']['f1-score'][0] >= best_score:
                best_score = mean_report['macro avg']['f1-score'][0]
                best_hyp_params = (units, learning_rate)
                best_report = mean_report

            print("\t\t--------------ACTUAL BEST: Units={}; Learning Rate={}--------------"
                  .format(best_hyp_params[0], best_hyp_params[1]))
            pretty_print_classification_report_dict(best_report)
            print("\t\t--------------ITERATION: Units={}; Learning Rate={}--------------"
                  .format(units, learning_rate))
            pretty_print_classification_report_dict(mean_report)

    with open(PKL_DIR + prefix, 'wb') as file:
        pickle.dump(all_clf_used, file)

    return best_hyp_params, best_report


In [ ]:
# IMPORTANTE -> ALGUNOS DE LOS PARÁMETROS DEL MODELO SON FIJADOS EN EL MÉTODO lstm_build_model del módulo
# classifier_utils.py ante problemas a la hora de clonar modelos neuronales de Keras.

lstm_dict = {
    cs.LSTM_SERIES_LENGTH: series_length,
    cs.LSTM_SEQUENCES_FRAGMENTER: 2,
    cs.LSTM_FITTED_LABELS_ENCODER: enc,
    cs.LSTM_ARGMAX_FUNCTION: argmax,
    cs.LSTM_N_CLASSES: 4,
    cs.LSTM_CLASS_WEIGHTS: class_weights,
    cs.LSTM_HYP_PARAM_EPOCHS: 50
}

lstm_randomized_search_cv(
    train.derived_data,
    train.derived_data_windows_per_serie,
    PKL_NAME,
    cv=5,
    lstm_dict=lstm_dict
)


# <font color=#cd0000> Randomized Search con múltiples ejecuciones en lugar de Validación Cruzada </font>
- Solo si tenemos pocos datos
- Ejecutaremos el mismo modelo sobre diferentes particiones del conjunto de datos original para observar su desempeño.

In [ ]:
# TODO

# <font color=#cd0000> Análisis de resultados </font>
- Según la búsqueda aleatoria de hiper-parámetros, la mejor combinación, es la de ``n_units`` = TODO y ``learning_rate`` = TODO:
    ```
    TODO
    ```
- Ahora vamos a visualizar la evolución de los resultados (25 resultados) para observar cómo avanza nuestra métrica objetivo -> Macro Average F1-Score.

In [ ]:
import pickle

all_reports = pickle.load(open(PKL_DIR + PKL_NAME, 'rb'))


In [ ]:
from utils.plot_utils import plot_score

macro_avg_f1_scores = dict(map(
    lambda z: (z, {'score': all_reports[z]['macro avg']['f1-score'][0],
                   'std': all_reports[z]['macro avg']['f1-score'][1]}),
    all_reports
))

plot_score(
    [macro_avg_f1_scores],
    ('n_units', 'learning_rate'),
    'LSTM',
    inverse=False,
    mode='score',
    in_same_graphic=True,
    accuracy_mode='accuracy',
    metric_name='Macro Average F1-Score'
)


In [ ]:
abnormal_recall_scores = dict(map(
    lambda z: (z, {'score': all_reports[z]['abnormal']['recall'][0],
                   'std': all_reports[z]['abnormal']['recall'][1]}),
    all_reports
))

normal_recall_scores = dict(map(
    lambda z: (z, {'score': all_reports[z]['normal']['recall'][0],
                   'std': all_reports[z]['normal']['recall'][1]}),
    all_reports
))

plot_score(
    [abnormal_recall_scores],
    ('n_units', 'learning_rate'),
    'LSTM',
    inverse=False,
    mode='score',
    in_same_graphic=True,
    accuracy_mode='accuracy',
    metric_name='Abnormal recall score'
)

plot_score(
    [normal_recall_scores],
    ('n_units', 'learning_rate'),
    'LSTM',
    inverse=False,
    mode='score',
    in_same_graphic=True,
    accuracy_mode='accuracy',
    metric_name='Normal recall score'
)


## <font color=#cd0000> Evaluación sobre el conjunto de validación </font>
- Vamos a llevar a cabo la evaluación final sobre el conjunto de validación (esto es lo que irá al apartado de ``Pruebas y Resultados`` de la memoria).

### <font color=#cd0000> Construcción del modelo </font>

In [ ]:
import keras as k
from keras.models import Sequential
from keras.layers import LSTM, Dense

nn = Sequential()

# Number of initial dimensions
nn.add(LSTM(units=12, dropout=.2, recurrent_dropout=.2))
# Number of Epilepsy's classes
nn.add(Dense(4, activation='sigmoid'))

In [ ]:
from keras.optimizers import RMSprop

metrics = [
    k.metrics.CategoricalAccuracy(name="ACC"),
    k.metrics.Precision(name='Prec'),
    k.metrics.Recall(name='Rec'),
    k.metrics.AUC(name='AUC')
]

nn.compile(optimizer=RMSprop(
    learning_rate=1e-4), loss='binary_crossentropy', metrics=metrics)
nn.build(input_shape=X_train.shape)


### <font color=#cd0000> Entrenamiento </font>

In [ ]:
import numpy as np
from keras import backend as K
from sklearn.metrics import confusion_matrix, classification_report

epochs = 10

history = nn.fit(X_train, y_train, epochs=epochs,
                 validation_data=(X_test, y_test))
nn.summary()
print('\n\n')

y_pred = argmax(nn.predict(X_test), 4)

show_metrics(history)

### <font color=#cd0000> Clasificación </font>

In [ ]:
y_real = enc.inverse_transform(y_test)
y_pred = enc.inverse_transform(y_pred)
print(confusion_matrix(y_real, y_pred))
print(classification_report(y_real, y_pred, zero_division=0))

# <font color=#cd0000> Conclusiones </font>
- TODO - Unas breves conclusiones sobre los resultados obtenidos (influencia de la codificación, ...)